In [2]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
import csv
with open('../Train/train_all_tasks.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    header = next(reader)
    rows = [row for row in reader]

In [5]:
header

['rewire_id', 'text', 'label_sexist', 'label_category', 'label_vector']

In [18]:
rows[:2]

[['sexism2022_english-7358',
  'Damn, this writing was pretty chaotic',
  'not sexist',
  'none',
  'none'],
 ['sexism2022_english-2367',
  'Yeah, and apparently a bunch of misogynistic virgins are the ones to turn a gay woman straight, lol',
  'not sexist',
  'none',
  'none']]

In [7]:
import re
import string
from emoji import demojize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words_nltk = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def preprocess(text):
  def rep(m):
    return ' '.join(re.split(r'(?=[A-Z])', m.group(1)))

  text = re.sub(r'#(\w+)', rep, text)

  def fin_word(word):
    if  word[0] == "'" and len(word) > 1 and word[1] not in ["s"]:
      return "'" + lemmatizer.lemmatize(word[1:].lower()) + "'"
    if word[0] == ':':
      return lemmatizer.lemmatize(word[1:].lower())
    return lemmatizer.lemmatize(word.lower())

  text = ' '.join([fin_word(word)
    for word in word_tokenize(demojize(text)) 
    if word.lower() not in ["url", "user"]
    and not all([ch in string.punctuation for ch in word])
    # and not word[0].isdigit() 
    and word != '’'
    and not word.lower() in stop_words_nltk])

  return text

# text = 'Thanks man, I know I’m a catch and I’m good looking just as we all are 💪 but I think I get too needy and am too available sometimes and these super attractive girls sniff it out'
# text = 'Thought: God took on flesh by being born to a woman (became incarnate). Perhaps that is why the Left is so obsessed with their Pink P😸😸😸y hats and menstrual blood and wombs, etc etc. They are yearning for the mother who points the way toward her Son.'
# text = "Honestly, I think I could get an ugly and/or 40+ one tomorrow, they can be desperate like that. Especially some drunk slut -- easy. But that's cheating, she must be attractive to me as well to make it count."
# text = "BIAS ALERT: 'Furious' NYT writer pens attack on white women | [URL]"
# text =  "That's very disappointing. Fire her. Now. #GabFam"
text = 'I said act like a whore. Also [URL] "A woman who has many casual sexual encounters or relationships." "A person who is regarded as willing to do anything to get a particular thing."'
preprocess(text)

'said act like whore also woman many casual sexual encounter relationship person regarded willing anything get particular thing'

### Split data

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split

# rows[0] - id
# rows[1] - text
# rows[2] - label
texts = np.array([preprocess(line[1]) for line in rows])
labels = np.array([1 if line[2] == "sexist" else 0 for line in rows], dtype='int')
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.20, random_state=42)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

def word_embedding(vectorizer):
  if vectorizer == 'tfidf':
    vectorizer = TfidfVectorizer(max_features = 1000)
    train_data = vectorizer.fit_transform(train_texts)
    test_data = vectorizer.transform(test_texts)
  elif vectorizer == 'w2v':
    def getVectorization(text, out_of_vocab):
      vects = []
      for word in text.split():
          try:
            vects.append(w2v.wv.get_vector(word))
          except:
            out_of_vocab = out_of_vocab + 1
              # print(word)
      return vects

    # dataset = [preprocess(text).split() for text in np.array(list(train_texts) + brownData)]
    dataset = [preprocess(text).split() for text in np.array(list(train_texts))]

    w2v = Word2Vec(sentences=dataset, size=1000, window=5, min_count=1, workers=16)
    w2v.save("word2vec.model")

    out_of_vocab = 0
    train_data = [np.mean(np.array(getVectorization(preprocess(text), out_of_vocab)), axis=0) for text in train_texts]
    test_data = [np.mean(np.array(getVectorization(preprocess(text), out_of_vocab)), axis=0) for text in test_texts]
    print(out_of_vocab)

  return train_data, test_data

In [10]:
train_tfidf, test_tfidf = word_embedding('tfidf')

In [11]:
train_w2v, test_w2v = word_embedding('w2v')

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0


In [12]:
train_data, test_data = train_tfidf.toarray(), test_tfidf.toarray()

In [15]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(train_data, train_labels)
print(np.mean(svc.predict(train_data) == train_labels))
print(np.mean(svc.predict(test_data) == test_labels))

0.8963392857142857
0.8210714285714286


In [16]:
train_data, test_data = np.array(train_w2v), np.array(test_w2v)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [17]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(train_data, train_labels)
print(np.mean(svc.predict(train_data) == train_labels))
print(np.mean(svc.predict(test_data) == test_labels))

ValueError: ignored